检查cuda版本型号，貌似10.2的还没适配cupy

In [0]:
!cat /usr/local/cuda/version.txt

CUDA Version 10.0.130


In [0]:
!nvidia-smi

Thu Mar  5 08:58:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

检查cuda是否有用

In [0]:
import cupy
cupy.cuda.device.get_device_id()

0

没用的话，需要安装10.0cuda或者重新分配gpu

In [0]:
'''
!apt-get update && sudo apt-get install -y build-essential git libgfortran3
!wget https://developer.nvidia.com/compute/cuda/10.0/Prod/local_installers/cuda-repo-ubuntu1804-10-0-local-10.0.130-410.48_1.0-1_amd64
!sudo dpkg -i cuda-repo-ubuntu1804-10-0-local-10.0.130-410.48_1.0-1_amd64.deb
!sudo apt-key add /var/cuda-repo-<version>/7fa2af80.pub
!sudo apt-get update
!sudo apt-get install cuda
'''

安装一些colab没有的库

In [0]:
!pip install ipdb
!pip install visdom
!pip3 install git+https://github.com/pytorch/tnt.git@master


In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('python3 -m pip install visdom')
get_ipython().system_raw('python3 -m visdom.server -port 6606 >> visdomlog.txt 2>&1 &')
get_ipython().system_raw('./ngrok http 6606 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


import visdom
vis = visdom.Visdom(port='6606', env='test')

--2020-03-05 09:00:06--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.73.183.190, 34.197.27.35, 34.228.134.34, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.73.183.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  6.59MB/s    in 2.0s    

2020-03-05 09:00:09 (6.59 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://2817cbed.ngrok.io


Setting up a new session...


In [0]:
from __future__ import division

import os
import xml.etree.ElementTree as ET

import cv2
import ipdb
import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image
# from wider import WIDER
from skimage import transform as sktsf
from torch.autograd import Variable
import time

import sys 
sys.path.append("/content/drive/My Drive/img_attack_with_attention") 
#import ipynb_importer

import attacks
from data.dataset import inverse_normalize
from data.dataset import pytorch_normalze
from data.util import read_image
from model import FasterRCNNVGG16
from trainer import BRFasterRcnnTrainer
from utils import array_tool as at
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

/content/drive/My Drive/img_attack_with_attention/model/utils/nms/non_maximum_suppression.py:15: UserWarning: 
    the python code for non_maximum_suppression is about 2x slow
    It is strongly recommended to build cython code:
    `cd model/utils/nms/; python3 build.py build_ext --inplace
  `cd model/utils/nms/; python3 build.py build_ext --inplace''')


In [0]:
VOC_BBOX_LABEL_NAMES = (
    'aeroplane',
    'bicycle',
    'bird',
    'boat',
    'bottle',
    'bus',
    'car',
    'cat',
    'chair',
    'cow',
    'diningtable',
    'dog',
    'horse',
    'motorbike',
    'person',
    'pottedplant',
    'sheep',
    'sofa',
    'train',
    'tvmonitor')

# VOC_BBOX_LABEL_NAMES = (
#     'n02691156',
#     'n02419796',
#     'n02131653',
#     'n02834778',
#     'n01503061',
#     'n02924116',
#     'n02958343',
#     'n02402425',
#     'n02084071',
#     'n02121808',
#     'n02503517',
#     'n02118333',
#     'n02510455',
#     'n02342885',
#     'n02374451',
#     'n02129165',
#     'n01674464',
#     'n02484322',
#     'n03790512',
#     'n02324045',
#     'n02509815',
#     'n02411705',
#     'n01726692',
#     'n02355227',
#     'n02129604',
#     'n04468005',
#     'n01662784',
#     'n04530566',
#     'n02062744',
#     'n02391049'
# )


In [0]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCdevkit_08-Jun-2007.tar

--2020-03-05 09:24:41--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/x-tar]
Saving to: ‘VOCtrainval_06-Nov-2007.tar’

VOCtrainval_06-Nov- 100%[===================>] 438.72M  5.22MB/s    in 90s     

2020-03-05 09:26:12 (4.88 MB/s) - ‘VOCtrainval_06-Nov-2007.tar’ saved [460032000/460032000]

--2020-03-05 09:26:13--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 451020800 (430M) [application/x-tar]
Saving to: ‘VOCtest_06-Nov-2007.tar’

VOCtest_06-Nov-2007 100%[===================

In [0]:
!tar xvf VOCtrainval_06-Nov-2007.tar
!tar xvf VOCtest_06-Nov-2007.tar
!tar xvf VOCdevkit_08-Jun-2007.tar

In [0]:
data_dir = '/content/VOCdevkit/VOC2007'
attacker_path = '/content/drive/My Drive/img_attack_with_attention/checkpoints/attack_03041223_2500.path'
save_path_HOME = '/content/drive/My Drive/img_attack_with_attention/sdd'


In [0]:
class VOCBboxDataset:
    """Bounding box dataset for PASCAL `VOC`_.
    .. _`VOC`: http://host.robots.ox.ac.uk/pascal/VOC/voc2012/
    The index corresponds to each image.
    When queried by an index, if :obj:`return_difficult == False`,
    this dataset returns a corresponding
    :obj:`img, bbox, label`, a tuple of an image, bounding boxes and labels.
    This is the default behaviour.
    If :obj:`return_difficult == True`, this dataset returns corresponding
    :obj:`img, bbox, label, difficult`. :obj:`difficult` is a boolean array
    that indicates whether bounding boxes are labeled as difficult or not.
    The bounding boxes are packed into a two dimensional tensor of shape
    :math:`(R, 4)`, where :math:`R` is the number of bounding boxes in
    the image. The second axis represents attributes of the bounding box.
    They are :math:`(y_{min}, x_{min}, y_{max}, x_{max})`, where the
    four attributes are coordinates of the top left and the bottom right
    vertices.
    The labels are packed into a one dimensional tensor of shape :math:`(R,)`.
    :math:`R` is the number of bounding boxes in the image.
    The class name of the label :math:`l` is :math:`l` th element of
    :obj:`VOC_BBOX_LABEL_NAMES`.
    The array :obj:`difficult` is a one dimensional boolean array of shape
    :math:`(R,)`. :math:`R` is the number of bounding boxes in the image.
    If :obj:`use_difficult` is :obj:`False`, this array is
    a boolean array with all :obj:`False`.
    The type of the image, the bounding boxes and the labels are as follows.
    * :obj:`img.dtype == numpy.float32`
    * :obj:`bbox.dtype == numpy.float32`
    * :obj:`label.dtype == numpy.int32`
    * :obj:`difficult.dtype == numpy.bool`
    Args:
        data_dir (string): Path to the root of the training data.
            i.e. "/data/image/voc/VOCdevkit/VOC2007/"
        split ({'train', 'val', 'trainval', 'test'}): Select a split of the
            dataset. :obj:`test` split is only available for
            2007 dataset.
        year ({'2007', '2012'}): Use a dataset prepared for a challenge
            held in :obj:`year`.
        use_difficult (bool): If :obj:`True`, use images that are labeled as
            difficult in the original annotation.
        return_difficult (bool): If :obj:`True`, this dataset returns
            a boolean array
            that indicates whether bounding boxes are labeled as difficult
            or not. The default value is :obj:`False`.
    """

    def __init__(self, data_dir, split='test',
                 use_difficult=False, return_difficult=False,
                 ):
        id_list_file = os.path.join(
            data_dir, 'ImageSets/Main/{0}.txt'.format(split))
        # id_list_file = os.listdir(data_dir)
        self.ids = [id_.strip() for id_ in open(id_list_file)]
        self.data_dir = data_dir
        self.use_difficult = use_difficult
        self.return_difficult = return_difficult
        self.label_names = VOC_BBOX_LABEL_NAMES
        self.save_dir = save_path_HOME + '/frcnn/'
        self.save_dir_adv = save_path_HOME + '/JPEGImages/'
        self.save_dir_perturb = save_path_HOME + '/frcnn_perturb/'

    def __len__(self):
        return len(self.ids)

    def preprocess(self, img, min_size=300, max_size=448):
        """Preprocess an image for feature extraction.
        The length of the shorter edge is scaled to :obj:`self.min_size`.
        After the scaling, if the length of the longer edge is longer than
        :param min_size:
        :obj:`self.max_size`, the image is scaled to fit the longer edge
        to :obj:`self.max_size`.
        After resizing the image, the image is subtracted by a mean image value
        :obj:`self.mean`.
        Args:
            img (~numpy.ndarray): An image. This is in CHW and RGB format.
                The range of its value is :math:`[0, 255]`.
             (~numpy.ndarray): An image. This is in CHW and RGB format.
                The range of its value is :math:`[0, 255]`.
        Returns:
            ~numpy.ndarray:
            A preprocessed image.
        """
        C, H, W = img.shape
        max_size = 400
        scale1 = min_size / min(H, W)
        scale2 = max_size / max(H, W)
        scale = min(scale1, scale2)
        img = img / 255.
        try:
            # img = sktsf.resize(img, (C, H * scale, W * scale), mode='reflect')
            img = sktsf.resize(img, (C, 300, 300), mode='reflect')
        except:
            ipdb.set_trace()
        # both the longer and shorter should be less than
        # max_size and min_size
        normalize = pytorch_normalze
        return normalize(img)

    def get_example(self, i):
        """Returns the i-th example.
        Returns a color image and bounding boxes. The image is in CHW format.
        The returned image is RGB.
        Args:
            i (int): The index of the example.
        Returns:
            tuple of an image and bounding boxes
        """
        id_ = self.ids[i]
        # print('id of img is:' + id_)
        anno = ET.parse(
            os.path.join(self.data_dir, 'Annotations', id_ + '.xml'))
        bbox = list()
        label = list()
        difficult = list()
        for obj in anno.findall('object'):
            # when in not using difficult split, and the object is
            # difficult, skipt it.
            # if not self.use_difficult and int(obj.find('difficult').text) == 1:
            #     continue

            # difficult.append(int(obj.find('difficult').text))
            bndbox_anno = obj.find('bndbox')
            # subtract 1 to make pixel indexes 0-based
            bbox.append([
                int(bndbox_anno.find(tag).text) - 1
                for tag in ('ymin', 'xmin', 'ymax', 'xmax')])
            name = obj.find('name').text.lower().strip()
            label.append(VOC_BBOX_LABEL_NAMES.index(name))
        bbox = np.stack(bbox).astype(np.float32)
        label = np.stack(label).astype(np.int32)
        # When `use_difficult==False`, all elements in `difficult` are False.
        difficult = np.array(np.zeros(label.shape), dtype=np.bool).astype(np.uint8)  # PyTorch don't support np.bool

        # # Load a image
        img_file = os.path.join(self.data_dir, 'JPEGImages', id_ + '.jpg')
        img = read_image(img_file, color=True)

        img = self.preprocess(img)
        img = torch.from_numpy(img)[None]
        # if self.return_difficult:
        #     return img, bbox, label, difficult
        return img, bbox, label, difficult, id_

    __getitem__ = get_example


In [0]:
def add_bbox(ax,bbox,label,score):
    for i, bb in enumerate(bbox):
        xy = (bb[1], bb[0])
        height = bb[2] - bb[0]
        width = bb[3] - bb[1]
        ax.add_patch(plt.Rectangle(
            xy, width, height, fill=False, edgecolor='red', linewidth=2))

        caption = list()
        label_names = list(VOC_BBOX_LABEL_NAMES) + ['bg']
        if label is not None and label_names is not None:
            lb = label[i]
            if not (-1 <= lb < len(label_names)):  # modfy here to add backgroud
                raise ValueError('No corresponding name is given')
            caption.append(label_names[lb])
        if score is not None:
            sc = score[i]
            caption.append('{:.2f}'.format(sc))

        if len(caption) > 0:
            ax.text(bb[1], bb[0],
                    ': '.join(caption),
                    style='italic',
                    bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 0})
    return ax

In [0]:
def img2jpg(img, img_suffix, quality):
    jpg_base = '/media/drive/ibug/300W_cropped/frcnn_adv_jpg/'
    img = img.transpose((1, 2, 0))
    img = Image.fromarray(img.astype('uint8'))
    if not os.path.exists(jpg_base):
        os.makedirs(jpg_base)
    jpg_path = jpg_base + img_suffix
    img.save(jpg_path, format='JPEG', subsampling=0, quality=quality)
    jpg_img = read_image(jpg_path)
    return jpg_img

In [0]:
if __name__ == '__main__':

    vis = visdom.Visdom(use_incoming_socket=True)

    layer_idx = 20
    _data = VOCBboxDataset(data_dir)
    faster_rcnn = FasterRCNNVGG16()
    faster_rcnn.eval()
    attacker = attacks.Blade_runner()
    attacker.load(attacker_path)
    attacker.eval()
    trainer = BRFasterRcnnTrainer(faster_rcnn, attacker, \
                                  layer_idx=layer_idx, attack_mode=True).cuda()
    trainer.load('/content/drive/My Drive/model_weights/fasterrcnn_12211511_0.701052458187_torchvision_pretrain.pth')
    quality_list = [100, 90, 80, 70, 60, 50, 40, 30, 20]
    threshold = [0.3]
    adv_det_list = []
    for quality in threshold:
        img_detected_total = 0
        adv_detected_total = 0
        img_object_total = 0
        total_distance = 0
        total_time = 0
        trainer.faster_rcnn.score_thresh = quality
        for i in range(len(_data)):
            if i == 100:
                print('===================================')
                print(str(i) + ' imgs have been generated')
                print('===================================')
                break

            img_detected = 0
            adv_detected = 0
            img_object = 0

            # img,im_path = _data[i]
            img, _, img_labels, _, img_id = _data[i]
            # if int(img_id.split('.')[0]) > 231:
            #     continue
            img_labels_lists = np.unique(img_labels)
            img = Variable(img.float().cuda())
            # im_path_clone = b = '%s' % im_path
            rois, roi_scores = faster_rcnn(img, flag=True)
            rois_num = len(rois)
            if rois_num < 300:
                print(i)
            im_path_clone = str(img_id)
            # save_path = _data.save_dir + 'f0rcnn' + im_path.split('/')[-1]
            save_path = _data.save_dir + im_path_clone.split('/')[-1] + '.jpg'
            save_path_adv = _data.save_dir_adv + im_path_clone.split('/')[-1] + '.jpg'
            save_path_perturb = _data.save_dir_perturb + 'frcnn_perturb_' + im_path_clone.split('/')[-1] + '.jpg'

            if not os.path.exists(_data.save_dir):
                os.makedirs(_data.save_dir)
            if not os.path.exists(_data.save_dir_adv):
                os.makedirs(_data.save_dir_adv)
            if not os.path.exists(_data.save_dir_perturb):
                os.makedirs(_data.save_dir_perturb)

            ori_img_ = inverse_normalize(at.tonumpy(img[0]))
            # _bboxes, _labels, _scores = trainer.faster_rcnn.predict([ori_img_],\
            #         new_score=quality, visualize=True)
            before = time.time()
            adv_img, perturb, distance = trainer.attacker.perturb(img, save_perturb=save_path_perturb, rois=rois, roi_scores=roi_scores)
            after = time.time()
            generate_time = after-before
            total_time = total_time + generate_time
            total_distance = total_distance + distance
            adv_img_ = inverse_normalize(at.tonumpy(adv_img[0]))
            perturb_ = inverse_normalize(at.tonumpy(perturb[0]))
            del adv_img, perturb, img
            perturb_ = perturb_.transpose((1, 2, 0))

            # 将图片从BGR转换为RGB格式进行保存
            perturb_RGB = cv2.cvtColor(perturb_, cv2.COLOR_BGR2RGB)
            ori_img_RGB = cv2.cvtColor(ori_img_.transpose((1, 2, 0)), cv2.COLOR_BGR2RGB)
            adv_img_RGB = cv2.cvtColor(adv_img_.transpose((1, 2, 0)), cv2.COLOR_BGR2RGB)

            cv2.imwrite(save_path_perturb, perturb_RGB)
            cv2.imwrite(save_path, ori_img_RGB)
            cv2.imwrite(save_path_adv, adv_img_RGB)
            print('The mean distance between ori and adv is %f' % (total_distance / i))

            #
            # adv_bboxes, adv_labels, adv_scores = trainer.faster_rcnn.predict([adv_img_], \
            #         new_score=quality, visualize=True, adv=True)

            print('generate adv for ', img_id)
        print('generate adv for all imgs')

Setting up a new session...


initialize network with normal


Setting up a new session...
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in double_scalars


The mean distance between ori and adv is inf
generate adv for  000001
The mean distance between ori and adv is 41.392279
generate adv for  000002
The mean distance between ori and adv is 36.062726
generate adv for  000003
The mean distance between ori and adv is 29.926106
generate adv for  000004
The mean distance between ori and adv is 31.363012
generate adv for  000006
The mean distance between ori and adv is 29.311218
generate adv for  000008
The mean distance between ori and adv is 29.677152
generate adv for  000010
The mean distance between ori and adv is 28.827661
generate adv for  000011
The mean distance between ori and adv is 26.993849
generate adv for  000013
The mean distance between ori and adv is 26.894072
generate adv for  000014
The mean distance between ori and adv is 26.040472
generate adv for  000015
The mean distance between ori and adv is 25.391285
generate adv for  000018
The mean distance between ori and adv is 25.737381
generate adv for  000022
The mean distance 

In [0]:
print(faster_rcnn)

In [0]:
print(attacker)